In [8]:
import os
# Find the latest version of spark 2.0  from http://www-us.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.0'
spark_version = 'spark-3.0.1'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www-us.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:3 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:5 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:8 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:13 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:14 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic/main Sources [1,688 kB]

In [9]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2020-11-15 03:35:12--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar.1’

postgresql-42.2.16. 100%[===================>] 979.38K  4.00MB/s    in 0.2s    

2020-11-15 03:35:12 (4.00 MB/s) - ‘postgresql-42.2.16.jar.1’ saved [1002883/1002883]



In [10]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("BigData-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrame

In [11]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Books_v1_01.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Books_v1_01.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22480053|R28HBXXO1UEVJT|0843952016|      34858117|          The Rising|           Books|          5|            0|          0|   N|                N|Great Twist on Zo...|I've known about ...| 2012-05-03|
|         US|   44244451| RZKRFS2UUMFFU|031088926X|     676347131|Sticky Faith Teen...|           Books|          5|    

### Create DataFrames to match tables

In [12]:
import pyspark.sql.functions as F
#from pyspark.sql.functions import to_date
# Read in the Review dataset as a DataFrame


In [13]:
# Create the customers_table DataFrame
# customers_df = df.groupby("customer_id").agg({""}).withColumnRenamed("", "customer_count")
customers_df = df.groupby("customer_id").agg(F.count("customer_id")).withColumnRenamed("count(customer_id)", "customer_count")
customers_df.show()

+-----------+--------------+
|customer_id|customer_count|
+-----------+--------------+
|   47098686|             1|
|    9973109|             1|
|   52477231|            24|
|   27568383|             1|
|   15519001|             2|
|   52130093|             2|
|   48794123|           102|
|   12453285|             1|
|   22824019|             1|
|   21087503|             9|
|   52553295|             4|
|   44143172|             1|
|   47844321|             3|
|   40575930|             1|
|   12657201|             1|
|   38674185|             1|
|   40325486|           166|
|   42645542|             2|
|   18768108|             5|
|   10546543|           361|
+-----------+--------------+
only showing top 20 rows



In [14]:
# Create the products_table DataFrame and drop duplicates. 
products_df = df.select(["product_id", "product_title"]).drop_duplicates()
products_df.show()

+----------+--------------------+
|product_id|       product_title|
+----------+--------------------+
|1419701630|Chuck Close: Face...|
|0451216954|Dark Lover (Black...|
|0552142395|My Feudal Lord: A...|
|1931514941|   Love Hina, Vol. 1|
|0615462219|The Great Pain De...|
|0805087605|What's Inside You...|
|1439197334|Far from Here: A ...|
|1413748260|       Plasma Dreams|
|1613469845|           Bolt Clan|
|B005IUSVS0|       If I Were You|
|0486451550|Architectura Nava...|
|0312426267|Golden Boy: Memor...|
|0425247414|Murder on Fifth A...|
|0375722203|Random House Webs...|
|1615390588|Colodin Project, ...|
|047023847X|HTML, XHTML and C...|
|1892785390|Continuing Care S...|
|1467996149|Far Away in the S...|
|0307886948|Jeannie Out of th...|
|159038668X|The Kingdom and t...|
+----------+--------------------+
only showing top 20 rows



In [15]:
# Create the review_id_table DataFrame.
# review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", "review_date"]).drop_duplicates() 
# Convert the 'review_date' column to a date datatype with to_date("review_date", 'yyyy-MM-dd').alias("review_date")
review_id_df = df.select(["review_id", "customer_id", "product_id", "product_parent", F.to_date("review_date", 'yyyy-MM-dd').alias("review_date")])
review_id_df.drop_duplicates()
review_id_df.show()

+--------------+-----------+----------+--------------+-----------+
|     review_id|customer_id|product_id|product_parent|review_date|
+--------------+-----------+----------+--------------+-----------+
|R28HBXXO1UEVJT|   22480053|0843952016|      34858117| 2012-05-03|
| RZKRFS2UUMFFU|   44244451|031088926X|     676347131| 2012-05-03|
|R2WAU9MD9K6JQA|   20357422|0615268102|     763837025| 2012-05-03|
|R36SCTKYTVPZPC|   13235208|1900869225|     785539232| 2012-05-03|
|R10BM6JUOJX27Q|   26301786|1565129938|      64646125| 2012-05-03|
| RCLZ5OKZNUSY4|   27780192|146854456X|     270349766| 2012-05-03|
|R1S65DJYEI89G4|   13041546|1118094514|     752141158| 2012-05-03|
|R3KQYBQOLYDETV|   51692331|0563521147|     729491316| 2012-05-03|
|R3QV8K7CSU8K2W|   23108524|0669444421|     261004015| 2012-05-03|
|R3W5A1WUGO5VQ0|   51692331|1897784457|     497876045| 2012-05-03|
|R20AQCY3FMBVN5|   49438248|0316738158|     691490916| 2012-05-03|
| R7KY8VL871MVL|   11818020|0738730440|     544176812| 2012-05

In [16]:
# Create the vine_table. DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"]).drop_duplicates()
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R100BPR3GPV9G0|          5|            0|          4|   N|                Y|
|R100FFVENVPN43|          3|            0|          2|   N|                N|
|R100J2UKIZXUKI|          5|            2|          3|   N|                N|
|R100OSDXZ6I2H1|          5|            0|          1|   N|                N|
|R100P46OVWWXLJ|          5|            8|          9|   N|                N|
|R100SKP4VNPS64|          4|           16|         31|   N|                N|
|R100ZB7HIXV0OZ|          4|            0|          0|   N|                N|
|R1019R5OTNGL84|          5|            1|          2|   N|                N|
|R101B0XO8UYAFV|          1|           28|         64|   N|                N|
|R101BBAUYRQKES|          3|            0|          0|   N|     

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [17]:
# Configure settings for RDS
mode = "append"
jdbc_url="jdbc:postgresql://dataviz.clzu7bsd0lkj.us-east-2.rds.amazonaws.com:5432/dataviz"
config = {"user":"postgres", 
          "password": "postgres", 
          "driver":"org.postgresql.Driver"}

In [21]:
# Write review_id_df to table in RDS
review_id_df.write.jdbc(url=jdbc_url, table='review_id_table', mode=mode, properties=config)

In [19]:
# Write products_df to table in RDS
# about 3 min
products_df.write.jdbc(url=jdbc_url, table='products_table', mode=mode, properties=config)

In [20]:
# Write customers_df to table in RDS
# 5 min 14 s
customers_df.write.jdbc(url=jdbc_url, table='customers_table', mode=mode, properties=config)

In [17]:
# Write vine_df to table in RDS
# 11 minutes
vine_df.write.jdbc(url=jdbc_url, table='vine_table', mode=mode, properties=config)